In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook
import random
import gurobipy as gp
from gurobipy import GRB
import matplotlib.colors as mcolors
from sklearn.manifold import MDS

import pickle

# For the randomly-generated points
NUM_POINTS = 500
NUM_CLUSTERS = 5
SQUARE_SIZE = 600
NUM_OUTLIERS = 30

# For the taxi data points
NUM_TAXI_CLUSTERS = 5
NUM_TAXI_OUTLIERS = 16

TOLERANCE = ((SQUARE_SIZE**2/NUM_CLUSTERS)**0.5)
print(TOLERANCE)

The below cell generates a set of NUM_POINTS random points, to be split into NUM_CLUSTERS clusters

In [ ]:
point_dict = {}
distances_dict = {}
possible_pairs = {}
np.random.seed(43)
random_points = SQUARE_SIZE*np.random.rand(2,NUM_POINTS)
for i in range(NUM_POINTS):
    point_dict[("point_"+str(i))] = (random_points[0][i],random_points[1][i])

In [ ]:
for i in point_dict.keys():
    temp_list = []
    
    for j in point_dict.keys():
        dist_x = point_dict[i][0] - point_dict[j][0]
        dist_y = point_dict[i][1] - point_dict[j][1]
        distances_dict[(i,j)] = (dist_x**2+dist_y**2)**0.5
        if (distances_dict[(i,j)] <= TOLERANCE):
            temp_list.append(j)
    possible_pairs[i] = temp_list

In [ ]:
plot = plt.scatter(random_points[0],random_points[1],c="green")
plt.xlabel("x")
plt.ylabel("y")

In [ ]:
def formulate(possible_pairs, distances_dict, NUM_CLUSTERS, NUM_OUTLIERS=0):
    m = gp.Model("clustering")
    pairs = {}
    isCenter = {}
    outlier = {}
    for i in possible_pairs.keys():
        isCenter[i] = m.addVar(vtype=GRB.BINARY, name = "isCenter[%s]" %i)
        outlier[i] = m.addVar(vtype=GRB.BINARY, name = "outlier[%s]" %i)
        for j in possible_pairs[i]:
            pairs[i,j] = m.addVar(vtype=GRB.BINARY, name = "pair{%s,%s}" % (i,j))

    for j in possible_pairs.keys():#constraint to define the isNotCenter variable
        m.addConstr(isCenter[j] <= sum(pairs[i,j] for i in possible_pairs[j])) #isCenter[j] is LEQ than sum of all pairs [i,j]
        for i in possible_pairs[j]:#to ensure that isCenter is 1 if any point has it as a center
            m.addConstr(isCenter[j] - pairs[i,j] >= 0)

    for i in possible_pairs.keys():
        # this constraint was changed to ensure that each point i is either part of at least one cluster, or declared an outlier
        m.addConstr(sum(pairs[i,j] for j in possible_pairs[i]) + outlier[i] >= 1) 
        m.addConstr(pairs[i,i] >= isCenter[i])

    m.addConstr(sum(isCenter[j] for j in possible_pairs.keys()) == NUM_CLUSTERS)
    m.addConstr(sum(outlier[i] for i in possible_pairs.keys()) <= NUM_OUTLIERS)

    m.setObjective(sum(distances_dict[i,j]*pairs[i,j] for i in possible_pairs.keys() for j in possible_pairs[i]), GRB.MINIMIZE)
    return m, pairs, isCenter, outlier

In [ ]:
# The code to display annotations on hover drew heavily from https://stackoverflow.com/a/47166787
def plot_output(isCenter, pairs, outlier, point_dict, labels=None, mark_outliers=True):
    if labels is None:
        labels = dict(point_dict)
        for label in labels:
            labels[label] = f'({str(point_dict[label][0])[:6]}, {str(point_dict[label][1])[:6]})'
            
    centerDict = {}
    for center in isCenter.keys():
        if isCenter[center].x == 1:
            centerDict[center] = []

    for pair in pairs.keys():
        if pairs[pair].x == 1:
            centerDict[pair[1]].append(point_dict[pair[0]])

    for center in centerDict.keys():
        centerDict[center] = dict(centerDict[center])

    outs = []
    for out in outlier:
        if outlier[out].x == 1:
            outs.append(point_dict[out])
            if mark_outliers:
                labels[out] = 'Outlier: ' + labels[out]
    
    x = []
    y = []
    c = []
            
    color_selection = list(range(len(centerDict)))
    OUTLIER_COLOR=len(centerDict)
    for i, center in enumerate(centerDict.keys()):
        current_color = color_selection[i]
        (keys,values) = zip(*centerDict[center].items())
        x = x + list(keys)
        y = y + list(values)
        c = c + [current_color] * len(keys)
    if len(outs) > 0:
        keys, values = tuple(zip(*outs))
        x = x + list(keys)
        y = y + list(values)
        c = c + [OUTLIER_COLOR] * len(keys)
        
    norm = plt.Normalize(0,len(centerDict))
    cmap = plt.cm.coolwarm

    fig,ax = plt.subplots()
    sc = plt.scatter(x,y,c=c, cmap=cmap, norm=norm)
    
    annot = ax.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points",
                    bbox=dict(boxstyle="round", fc="w"),
                    arrowprops=dict(arrowstyle="->"))
    
    annot.set_visible(False)
    annot.set_wrap(True)
    
    inv_map = {v: k for k, v in point_dict.items()}
    
    def update_annot(ind):
        pos = sc.get_offsets()[ind["ind"][0]]
        annot.xy = pos
        text = str(labels[inv_map[tuple(pos)]])
        
        annot.set_text(text)
        annot.get_bbox_patch().set_facecolor(cmap(norm(c[ind["ind"][0]])))


    def hover(event):
        vis = annot.get_visible()
        if event.inaxes == ax:
            cont, ind = sc.contains(event)
            if cont:
                update_annot(ind)
                annot.set_visible(True)
                fig.canvas.draw_idle()
            else:
                if vis:
                    annot.set_visible(False)
                    fig.canvas.draw_idle()
                    
    fig.canvas.mpl_connect("motion_notify_event", hover)

    plt.show()

In [ ]:
# Formulation without outliers
m, pairs, isCenter, outlier = formulate(possible_pairs, distances_dict, NUM_CLUSTERS)
# Uncomment the line below to suppress Gurobi's output
# m.Params.LogToConsole = 0
m.optimize()

In [ ]:
plot_output(isCenter, pairs, outlier, point_dict)

In [ ]:
# solve the k-medians problem while allowing NUM_OUTLIERS points to go un-classified
m, pairs, isCenter, outlier = formulate(possible_pairs, distances_dict, NUM_CLUSTERS, NUM_OUTLIERS)
# Uncomment the line below to suppress Gurobi's output
# m.Params.LogToConsole = 0
m.optimize()

In [ ]:
plot_output(isCenter, pairs, outlier, point_dict)

In [ ]:
# Now, trying the above code on the taxi data.
# Locally, I copied the taxi_count_dict.pickle file from the MST clustering lab.
# These are number of rides hailed in 15? minute intervals for every day of one year.
# See the other lab for more info I guess

with open('data/taxi_count_dict.pickle', 'rb') as handle:
    taxi_counts = pd.DataFrame(pickle.load(handle))
print(taxi_counts.loc[0:6])

In [ ]:
len(taxi_counts)

In [ ]:
# preprocess data again
point_dict = {}
distances_dict = {}
possible_pairs = {}
for i in range(len(taxi_counts)):
    point_dict[("day_"+str(i))] = taxi_counts.loc[i]["count_vector"]

In [ ]:
for i in point_dict.keys():
    temp_list = []
    for j in point_dict.keys():
        distances_dict[(i,j)] = np.linalg.norm(np.array(point_dict[i]) - np.array(point_dict[j]), ord=1)
        if True:#(distances_dict[(i,j)] <= TOLERANCE): # Not sure what tolerance should be, so ignoring. If it is too slow, consider adding some tolerance.
            temp_list.append(j)
    possible_pairs[i] = temp_list

In [ ]:
# Compute clustering for the taxi dataset
NUM_OUTLIERS = NUM_TAXI_OUTLIERS
NUM_CLUSTERS = NUM_TAXI_CLUSTERS

m, pairs, isCenter, outlier = formulate(possible_pairs, distances_dict, NUM_CLUSTERS, NUM_OUTLIERS)
# Uncomment the line below to suppress Gurobi's output
# m.Params.LogToConsole = 0
m.optimize()

In [ ]:
# TODO: The above cell took ~ 15 seconds for me to run. 
# It may be worth seeing how long it takes if re-implemented in or-tools with their free MIP solver.


In [ ]:
centerDict = {}
for center in isCenter.keys():
    if isCenter[center].x == 1:
        centerDict[center] = []
        
for pair in pairs.keys():
    if pairs[pair].x == 1:
        centerDict[pair[1]].append(pair[0])
        
outs = []
for out in outlier:
    if outlier[out].x == 1:
        outs.append(out)
        
# A little bit to try to visualize the output.
# Indicates information about each of the outliers (month,day,day-of-week)
# and the same information about all the days, grouped by clusters.
    
print("OUTLIERS")
for day in outs:
    entry = taxi_counts.loc[int(day[4:])]
    print(day, "%s,%s:  %s" % (entry["m"],entry["d"],entry["weekday"]))

for center in centerDict:
    print("")
    print("CLUSTER")
    for day in centerDict[center]:
        entry = taxi_counts.loc[int(day[4:])]
        print(day, "%s,%s:  %s" % (entry["m"],entry["d"],entry["weekday"]))

In [ ]:
# Visualization.
# We want to embed the high-dimensional points into a 2-dimensional space so we can do a scatter plot.

# Here is an outline of what hellinger_manifold_mds does:
# - treat the points as probability distributions (by normalizing)
# - find the pairwise hellinger distance between them
# - use "multidimensional scaling" to embed into two dimensions (See https://scikit-learn.org/stable/modules/generated/sklearn.manifold.MDS.html)

# Possible steps 
# - Use some other standard clustering metrics to compare this clustering result against the MST clustering results.

In [ ]:
# Computes the Hellinger distances of iterables of numbers x and y
def hellinger(x, y):
    # Normalize x, y
    x = list(x)
    y = list(y)
    for a in (x,y):
        s = sum(a)
        for i in range(len(a)):
            a[i] = a[i] / s
    # Compute Hellinger distance
    squares = []
    for x_i, y_i in zip(x, y):
        squares.append((math.sqrt(x_i) - math.sqrt(y_i)) ** 2)
    return sum(squares) / math.sqrt(2)  

In [ ]:
# Visualization method using sklearn.manifold.MDS with Hellinger distance
# Uses sklearn's implementation of the SMACOF algorithm to scale point_dict to two dimensions using 
# the Hellinger distance between points
def hellinger_manifold_mds(point_dict, distances_dict):
    # Set up the hellinger distance matrix
    n = len(point_dict)
    D = np.zeros((n,n))
    for tup in distances_dict.keys():
        i = int(tup[0][len('day_'):])
        j = int(tup[1][len('day_'):])
        D[i,j] = hellinger(point_dict[tup[0]], point_dict[tup[1]])
    mds = MDS(dissimilarity='precomputed')
    return mds.fit_transform(D)

In [ ]:
# Visualization method using sklearn.manifold.MDS with Euclidean distance
# Uses sklearn's implementation of the SMACOF algorithm to scale point_dict to two dimensions using 
# the Euclidean distance between points
def euclidean_manifold_mds(point_dict, distances_dict):
    X = np.row_stack(tuple(np.array(point_dict[i]) for i in point_dict))
    mds = MDS()
    return mds.fit_transform(X)

In [ ]:
# Alternate visualization method
# Implements the classical MDS algorithm outlined in https://en.wikipedia.org/wiki/Multidimensional_scaling
def classical_mds(point_dict, distances_dict):
    DIM = 2 # The dimension to scale down to
    n = len(point_dict)
    # 1: Set up the squared proximity matrix
    D = np.zeros((n,n))
    for tup in distances_dict.keys():
        i = int(tup[0][len('day_'):])
        j = int(tup[1][len('day_'):])
        D[i,j] = distances_dict[tup]**2

    # 2: Apply double-centering
    C = np.identity(n) - 1/n * np.ones((n,n))
    B = -1/2 * C.dot(D).dot(C)

    # 3: Get the DIM largest eigenvalues and corresponding eigenvectors
    w, v = np.linalg.eig(B)
    dim_largest = sorted(w, reverse=True)[:DIM]
    wlist = w.tolist()
    w_indices = [wlist.index(i) for i in dim_largest]

    # 4: Get the new coordinates matrix, X
    return np.column_stack(tuple(v[:,i] for i in w_indices)).dot(np.diag(np.array([i**(1/2) for i in dim_largest])))

In [ ]:
# Choose the MDS algorithm here. Uncomment one of the following
# X = hellinger_manifold_mds(point_dict, distances_dict)
X = euclidean_manifold_mds(point_dict, distances_dict)
# X = classical_mds(point_dict, distances_dict)

In [ ]:
# Override each element of point_dict with the dimension change
new_point_dict = point_dict.copy()
for i in new_point_dict:
    j = int(i[len('day_'):])
    new_point_dict[i] = tuple(X[j])

In [ ]:
labels = {}
days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
for i in new_point_dict:
    j = int(i[len('day_'):])
    entry = taxi_counts.loc[j]
    month = entry["m"]
    day = entry["d"]
    weekday = entry["weekday"]
    labels[i] = f'{days[weekday]}, {months[month-1]} {day}'

In [ ]:
# Clustering plot
plot_output(isCenter, pairs, outlier, new_point_dict, dict(labels))

In [ ]:
# Plot colored by weekdays
class xObj:
    def __init__(self, x):
        self.x = x
        
days = {i: None for i in range(7)}
isCenter = {}
pairs = {}
for i in new_point_dict:
    j = int(i[len('day_'):])
    entry = taxi_counts.loc[j]
    month = entry["m"]
    day = entry["d"]
    weekday = entry["weekday"]
    x = new_point_dict[i][0]
    y = new_point_dict[i][1]
    if days[weekday] is None:
        days[weekday] = i
        isCenter[i] = xObj(1)
    else:
        isCenter[i] = xObj(0)
    for k in days:
        if days[k] is None:
            continue
        pair = (i, days[k])
        val = 1 if k == weekday else 0
        pairs[pair] = xObj(val)

plot_output(isCenter, pairs, {}, new_point_dict, dict(labels))